# Demo of `create_climpred_data`

This demo demonstrates how you setup your raw output from a climate model to match `climpred`'s expectations.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import climpred

In [2]:
from climpred.create_climpred_data import load_hindcast, climpred_preprocess_internal, get_path

Assuming your raw model output is stored in multiple files per member and initialization, `load_hindcast` is a nice wrapper function based on `get_path` designed for the output format of `MPI-ESM` to aggregated all hindcast output into one file as expected by `climpred`.

The basic idea is to look over the output of all members and concatinate, then loop over all initializations and concatinate. Before concatination, it is important to align the `time` dimension.

To reduce the data size, use the `preprocess` function wisely in combination with `climpred_preprocess_internal`, e.g. additionally extracting only a certain region or only few variables for a multi-variable input file as in MPI-ESM standard output.

In [3]:
get_path??

Signature:
get_path(
    dir_base_experiment='/work/bm1124/m300086/CMIP6/experiments',
    member=1,
    init=1960,
    model='hamocc',
    output_stream='monitoring_ym',
    timestr='*1231',
    ending='nc',
)
Source:   
def get_path(
    dir_base_experiment='/work/bm1124/m300086/CMIP6/experiments',
    member=1,
    init=1960,
    model='hamocc',
    output_stream='monitoring_ym',
    timestr='*1231',
    ending='nc',
):
    """Get the path of a file of for MPI-ESM standard output file names and directory.

    Args:
        dir_base_experiment (str): Path of experiments folder. Defaults to "/work/bm1124/m300086/CMIP6/experiments".
        member (int): `member`. Defaults to 1.
        init (init): `init`. Defaults to 1960.
        model (str): submodel name. Defaults to "hamocc". Allowed: ['echam6', 'jsbach', 'mpiom', 'hamocc'].
        output_stream (str): output_stream name. Defaults to "monitoring_ym". Allowed: ['data_2d_mm', 'data_3d_ym', 'BOT_mm', ...]
        timestr (str): ti

In [4]:
climpred_preprocess_internal??

Signature: climpred_preprocess_internal(ds, lead_offset=1, time_dim='time')
Source:   
def climpred_preprocess_internal(ds, lead_offset=1, time_dim='time'):
    """CMIP6 DCPP preprocessing before the aggreatations of intake-esm happen."""
    # set time_dim to integers starting at lead_offset
    ds[time_dim] = np.arange(lead_offset, lead_offset + ds[time_dim].size)
    return ds
File:      /work/mh0727/m300524/miniconda3/envs/climpred-dev/lib/python3.6/site-packages/climpred-1.2.0.post6+dirty-py3.6.egg/climpred/create_climpred_data.py
Type:      function


In [5]:
load_hindcast??

Signature:
load_hindcast(
    inits=range(1961, 1965),
    members=range(1, 3),
    preprocess=None,
    lead_offset=1,
    parallel=True,
    engine=None,
    **get_path_kwargs,
)
Source:   
def load_hindcast(
    inits=range(1961, 1965),
    members=range(1, 3),
    preprocess=None,
    lead_offset=1,
    parallel=True,
    engine=None,
    **get_path_kwargs,
):
    """Load multi-member, multi-initialization hindcast experiment into one xr.Dataset compatible with `climpred`.

    Args:
        inits (list, array): List of initializations to be loaded. Defaults to range(1961, 1965).
        members (list, array): List of initializations to be loaded. Defaults to range(1, 3).
        preprocess (function): `preprocess` function accepting and returning xr.Datasets only to be passed to `xr.open_dataset`. Defaults to None.
        lead_offset (int): How many leads to throw away. Defaults to 1.
        parallel (bool): passed to `xr.open_dataset`. Defaults to True.
        engine (str): pa

In [6]:
def preprocess_1var(ds, v="global_primary_production"):
    return ds[v].to_dataset(name=v).squeeze()

In [7]:
ds = load_hindcast(preprocess=preprocess_1var).load()
ds.coords

Processing init 1961 ...
Processing init 1962 ...
Processing init 1963 ...
Processing init 1964 ...


Coordinates:
    lat      float64 0.0
    lon      float64 0.0
    depth    float64 0.0
  * lead     (lead) int64 1 2 3 4 5 6 7 8 9 10
  * member   (member) int64 1 2
  * init     (init) int64 1961 1962 1963 1964

In [8]:
climpred.prediction.compute_perfect_model(ds, ds.rename({'lead':'time'}))

<xarray.Dataset>
Dimensions:                    (lead: 10)
Coordinates:
    lat                        float64 0.0
    lon                        float64 0.0
    depth                      float64 0.0
  * lead                       (lead) int64 1 2 3 4 5 6 7 8 9 10
Data variables:
    global_primary_production  (lead) float64 -0.1276 0.593 ... 0.09196 0.5038
Attributes:
    prediction_skill:              calculated by climpred https://climpred.re...
    skill_calculated_by_function:  compute_perfect_model
    number_of_initializations:     4
    number_of_members:             2
    metric:                        pearson_r
    comparison:                    m2e
    units:                         None
    created:                       2020-01-06 15:49:17

# `intake-esm` for cmorized output

In case you have access to cmorized output of CMIP experiments, consider using `intake-esm`. With the `preprocess` function you can align the `time` dimension of the output. Finally, `climpred_preprocess_post` only renames.

In [9]:
from climpred.create_climpred_data import climpred_preprocess_post, climpred_preprocess_internal

In [10]:
import intake

In [11]:
col_url = "/home/mpim/m300524/intake-esm-datastore/catalogs/mistral-cmip6.json"
col = intake.open_esm_datastore(col_url)

In [18]:
col.df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
0,AerChemMIP,HAMMOZ-Consortium,MPI-ESM-1-2-HAM,ssp370-lowNTCF,r1i1p1f1,Lmon,npp,gn,NaN,v20190627,203501-205412,/work/ik1017/CMIP6/data/CMIP6/AerChemMIP/HAMMO...
1,AerChemMIP,HAMMOZ-Consortium,MPI-ESM-1-2-HAM,ssp370-lowNTCF,r1i1p1f1,Lmon,npp,gn,NaN,v20190627,201501-203412,/work/ik1017/CMIP6/data/CMIP6/AerChemMIP/HAMMO...
2,AerChemMIP,HAMMOZ-Consortium,MPI-ESM-1-2-HAM,ssp370-lowNTCF,r1i1p1f1,Lmon,npp,gn,NaN,v20190627,205501-205512,/work/ik1017/CMIP6/data/CMIP6/AerChemMIP/HAMMO...
3,AerChemMIP,HAMMOZ-Consortium,MPI-ESM-1-2-HAM,ssp370-lowNTCF,r1i1p1f1,Lmon,tsl,gn,NaN,v20190627,205501-205512,/work/ik1017/CMIP6/data/CMIP6/AerChemMIP/HAMMO...
4,AerChemMIP,HAMMOZ-Consortium,MPI-ESM-1-2-HAM,ssp370-lowNTCF,r1i1p1f1,Lmon,tsl,gn,NaN,v20190627,201501-203412,/work/ik1017/CMIP6/data/CMIP6/AerChemMIP/HAMMO...


In [19]:
col.df.columns

Index(['activity_id', 'institution_id', 'source_id', 'experiment_id',
       'member_id', 'table_id', 'variable_id', 'grid_label', 'dcpp_init_year',
       'version', 'time_range', 'path'],
      dtype='object')

In [20]:
# load 2 members for 2 inits from one model
query = dict(experiment_id=[
    'dcppA-hindcast'], table_id='Amon', member_id=['r1i1p1f1', 'r2i1p1f1'], dcpp_init_year=[1970, 1971],
    variable_id='tas', source_id='MPI-ESM1-2-HR')
cat = col.search(**query)
cdf_kwargs = {'chunks': {'time': 12}, 'decode_times': False}

In [13]:
def preprocess(ds):
    # extract tiny spatial and temporal subset
    ds = ds.isel(lon=[50, 51, 52], lat=[50, 51, 52],
                 time=np.arange(12 * 2))
    # make time dim identical
    ds = climpred_preprocess_internal(ds)
    return ds

In [14]:
dset_dict = cat.to_dataset_dict(
    cdf_kwargs=cdf_kwargs, preprocess=preprocess)
# get first dict value
ds = dset_dict[list(dset_dict.keys())[0]].load()
ds.coords

Progress: |███████████████████████████████████████████████████████████████████████████████| 100.0% 

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'
             
--> There are 1 group(s)


Coordinates:
    height          float64 2.0
  * lon             (lon) float64 46.88 47.81 48.75
  * dcpp_init_year  (dcpp_init_year) float64 1.97e+03 1.971e+03
  * time            (time) int64 1 2 3 4 5 6 7 8 9 ... 17 18 19 20 21 22 23 24
  * lat             (lat) float64 -42.55 -41.61 -40.68
  * member_id       (member_id) <U8 'r1i1p1f1' 'r2i1p1f1'

In [15]:
ds = climpred_preprocess_post(ds)
ds.coords

Coordinates:
    height   float64 2.0
  * lon      (lon) float64 46.88 47.81 48.75
  * init     (init) float64 1.97e+03 1.971e+03
  * lead     (lead) int64 1 2 3 4 5 6 7 8 9 10 ... 15 16 17 18 19 20 21 22 23 24
  * lat      (lat) float64 -42.55 -41.61 -40.68
  * member   (member) <U8 'r1i1p1f1' 'r2i1p1f1'

In [16]:
climpred.prediction.compute_perfect_model(ds, ds.rename({'lead':'time'}))

/work/mh0727/m300524/miniconda3/envs/climpred-dev/lib/python3.6/site-packages/xskillscore/core/np_deterministic.py:182: RuntimeWarning: invalid value encountered in true_divide
  r = r_num / r_den


<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 3, lead: 24, lon: 3)
Coordinates:
    height     float64 2.0
  * lon        (lon) float64 46.88 47.81 48.75
  * lead       (lead) int64 1 2 3 4 5 6 7 8 9 10 ... 16 17 18 19 20 21 22 23 24
  * lat        (lat) float64 -42.55 -41.61 -40.68
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (lead, bnds) float64 nan nan nan nan nan ... nan nan nan nan nan
    lat_bnds   (lead, lat, bnds) float64 nan nan nan nan nan ... nan nan nan nan
    lon_bnds   (lead, lon, bnds) float64 nan nan nan nan nan ... nan nan nan nan
    tas        (lead, lat, lon) float32 0.933643 0.88438606 ... -0.8581106
Attributes:
    external_variables:            areacella
    table_id:                      Amon
    intake_esm_varname:            tas
    source:                        MPI-ESM1.2-HR (2017): \naerosol: none, pre...
    tracking_id:                   hdl:21.14100/f41d2fe5-bb1c-49d0-8afa-0cb9e...
    license:                       CMIP6 model data produced by MPI-M is lice...
    grid:                          spectral T127; 384 x 192 longitude/latitude
    data_specs_version:            01.00.30
    forcing_index:                 1
    experiment_id:                 dcppA-hindcast
    table_info:                    Creation Date:(09 May 2019) MD5:e6ef8ececc...
    mip_era:                       CMIP6
    frequency:                     mon
    realm:                         atmos
    initialization_index:          1
    variable_id:                   tas
    cmor_version:                  3.5.0
    branch_method:                 standard lagged initialization
    product:                       model-output
    title:                         MPI-ESM1-2-HR output prepared for CMIP6
    history:                       2019-09-06T14:20:04Z ; CMOR rewrote data t...
    source_type:                   AOGCM
    experiment:                    hindcast initialized based on observations...
    Conventions:                   CF-1.7 CMIP-6.2
    grid_label:                    gn
    contact:                       cmip6-mpi-esm@dkrz.de
    references:                    MPI-ESM: Mauritsen, T. et al. (2019), Deve...
    physics_index:                 1
    nominal_resolution:            100 km
    institution:                   Max Planck Institute for Meteorology, Hamb...
    source_id:                     MPI-ESM1-2-HR
    project_id:                    CMIP6
    activity_id:                   DCPP
    institution_id:                MPI-M
    prediction_skill:              calculated by climpred https://climpred.re...
    skill_calculated_by_function:  compute_perfect_model
    number_of_initializations:     2
    number_of_members:             2
    metric:                        pearson_r
    comparison:                    m2e
    units:                         None
    created:                       2020-01-06 15:49:35